# 타이타닉 모델 생성

In [1]:
import pandas as pd 
import joblib
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
df_train = pd.read_csv('../static/data/titanic_train.csv')
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,0,24.000000,0,0,8.8500,7,3
1,0,3,1,29.699118,8,2,69.5500,7,3
2,1,2,0,30.000000,0,0,12.3500,7,2
3,0,3,1,16.000000,4,1,39.6875,7,3
4,1,1,0,36.000000,0,2,71.0000,1,3


In [3]:
X_train = df_train.iloc[:, 1:].values
y_train = df_train.iloc[:, 0].values
X_train.shape, y_train.shape

((668, 8), (668,))

In [4]:
df_test = pd.read_csv('../static/data/titanic_test.csv')
df_test.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,0,20.0,0,0,8.6625,7,3
1,0,3,1,22.0,0,0,7.8958,7,3
2,0,3,1,33.0,0,0,7.8958,7,0
3,0,3,0,14.0,0,0,7.8542,7,3
4,0,3,0,39.0,1,5,31.2750,7,3


In [5]:
X_test = df_test.iloc[:, 1:].values
y_test = df_test.iloc[:, 0].values
X_test.shape, y_test.shape

((223, 8), (223,))

In [6]:
scaler = MinMaxScaler()
scaler.fit(df_train.iloc[:, :-1])

MinMaxScaler()

In [7]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((668, 8), (223, 8))

In [8]:
joblib.dump(scaler, '../static/model/titanic_scaler.pkl')

['../static/model/titanic_scaler.pkl']

### 1. Logistic Regression

In [9]:
lr_clf = LogisticRegression()
lr_clf.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [16]:
params = {
    #'C': [0.1, 1, 5, 10]
    #'C': [6, 8, 10, 12, 14]
    'C': [8, 9, 10, 11, 12]
}

grid_cv = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_)

최고 평균 정확도: 0.8039
최적 파라미터: {'C': 10}


In [17]:
best_lr = grid_cv.best_estimator_
pred = best_lr.predict(X_test_scaled)
accuracy_score(y_test, pred)

0.7757847533632287

In [18]:
joblib.dump(best_lr, '../static/model/titanic_lr.pkl')

['../static/model/titanic_lr.pkl']

### 2. SVM

In [19]:
sv_clf = SVC()
sv_clf.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [22]:
params = {
    #'C': [0.1, 1, 5, 10]
    #'C': [6, 8, 10, 12, 14]
    'C': [13, 14, 15, 16, 17, 18]
}

grid_cv = GridSearchCV(sv_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_)

최고 평균 정확도: 0.7605
최적 파라미터: {'C': 17}


In [23]:
best_sv = grid_cv.best_estimator_
pred = best_sv.predict(X_test_scaled)
accuracy_score(y_test, pred)

0.7533632286995515

In [24]:
joblib.dump(best_lr, '../static/model/titanic_sv.pkl')

['../static/model/titanic_sv.pkl']

### 3. Random Forest

In [25]:
rf_clf = RandomForestClassifier()
rf_clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [27]:
params = {
    #'max_depth': [2, 4, 6, 8],
    'max_depth': [7, 8, 9],
    #'min_samples_split': [1, 2, 3]
    'min_samples_split': [2, 3, 4]
}

grid_cv = GridSearchCV(rf_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_)

최고 평균 정확도: 0.8308
최적 파라미터: {'max_depth': 7, 'min_samples_split': 3}


In [28]:
best_rf = grid_cv.best_estimator_
pred = best_rf.predict(X_test_scaled)
accuracy_score(y_test, pred)

0.8251121076233184

In [29]:
joblib.dump(best_rf, '../static/model/titanic_rf.pkl')

['../static/model/titanic_rf.pkl']

### Test

In [30]:
index = 10
test_data = (df_test.iloc[index, :-1].values)
test_data

array([ 0.,  2.,  1., 42.,  0.,  0., 13.,  7.])

In [31]:
test_data = test_data.reshape(1,-1)
test_data

array([[ 0.,  2.,  1., 42.,  0.,  0., 13.,  7.]])

In [32]:
new_scaler = joblib.load('../static/model/titanic_scaler.pkl')

In [33]:
test_scaled = new_scaler.transform(test_data)
test_scaled

array([[0.        , 0.5       , 1.        , 0.52050473, 0.        ,
        0.        , 0.02537431, 0.875     ]])

In [35]:
label = df_test.iloc[index, 0]
label

0

In [36]:
pred_lr = best_lr.predict(test_data)
pred_sv = best_sv.predict(test_data)
pred_rf = best_rf.predict(test_data)

In [37]:
label, pred_lr[0], pred_sv[0], pred_rf[0]

(0, 0, 1, 0)

In [39]:
df_test.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,0,20.0,0,0,8.6625,7,3
1,0,3,1,22.0,0,0,7.8958,7,3
2,0,3,1,33.0,0,0,7.8958,7,0
3,0,3,0,14.0,0,0,7.8542,7,3
4,0,3,0,39.0,1,5,31.2750,7,3


In [40]:
tmp = df_test.iloc[index, 1:]
data =  []
int_index_list = [0, 1, 3, 4, 6, 7]
for i in range(8):
    if i in int_index_list:
        data.append(int(tmp[i]))
    else:
        data.append(tmp[i])
data

[2, 1, 42.0, 0, 0, 13.0, 7, 3]

In [41]:
a = dict(zip(df_test.columns[1:], data))
a

{'Pclass': 2,
 'Sex': 1,
 'Age': 42.0,
 'SibSp': 0,
 'Parch': 0,
 'Fare': 13.0,
 'Cabin': 7,
 'Embarked': 3}